In [2]:
from typing import List

In [1]:
import numpy as np

In [ ]:
#creating a simple neurone class Neuron:
class Neurone:
    #constructor
    def __init__ (self, num_inputs, activation = 'sigmoid'):
        self.activation_type = activation
        self.num_inputs = num_inputs
        self.weights = np.random.uniform(-1,1,num_inputs)
        self.bias = 0
        self.last_input = None
        self.last_pre_activation_output = None
        self.last_output = None
    
    def calculate_weighted_sum(self, inputs):
        weighted_sum = np.dot(self.weights, inputs)+self.bias
        return weighted_sum

    def forward(self, inputs):
        self.last_input = inputs.copy()
        weighted_sum = self.calculate_weighted_sum(inputs)
        self.last_pre_activation_output= weighted_sum
        if self.activation_type == 'sigmoid':
            last_output =  1/(1+np.exp(weighted_sum))
        elif self.activation_type == 'relu':
            last_output = max(0,weighted_sum)
        elif self.activation_type == 'tanh':
            last_output = np.tanh(weighted_sum)
        else:
            raise NotImplementedError(f"Activation function '{self.activation_type}' is not available.")
        self.last_output = last_output
        return last_output
    
    #this helps us to determine how much this neurone matters, if we change the raw output by changing the weights and inputs, will it affect its final output or it will be same? This is why we are finding derivative of the activation function wrt the raw output value. we need to find the rate of change of the function output with the raw output
    def activation_derivative(self, output_before_activation):
        if self.activation_type=='sigmoid':
            output = 1/(1+np.exp(-output_before_activation))
            return output * (1- output)
        elif self.activation_type == 'relu':
            return 1.0 if output_before_activation > 0 else 0.0
        elif self.activation_type == 'tanh': 
            return 1- np.tanh(output_before_activation)**2
        else:
            raise NotImplementedError(f"No derivative for '{self.activation_type}'")\
                
    def backward(self, delta, learning_rate):
        upstream = self.weights * delta   # ∂L/∂x = w(old) * δ

        gradient_weights = delta * self.last_input
        gradient_bias = delta 
        self.weights -= learning_rate * gradient_weights
        self.bias -= learning_rate * gradient_bias
        
        return upstream
        
        
        
        

In [ ]:
class Layer:
    def __init__(self, num_neurones, num_inputs, activation_type = 'sigmoid'):
        self.num_neurones = num_neurones
        self.neurones : List[Neurone] = []
        for _ in range(num_neurones):
            neurone = Neurone(num_inputs,activation_type)
            self.neurones.append(neurone)
    
    def forward(self, inputs):
        outputs = []
        for neurone in self.neurones:
            outputs.append(neurone.forward(inputs))
        return outputs
    
    def backward(self, deltas, learning_rate):
        
        upstream_deltas = np.zeros(self.neurones[0].num_inputs)
        
        for delta, neurone in zip(deltas, self.neurones):
            contribution = neurone.backward(delta, learning_rate)
            upstream_deltas += contribution
        return upstream_deltas

In [ ]:
class NeuralNetwork:
    def __init__(self, num_inputs, loss_function_type='mean_squared_error'):
        self.num_inputs = num_inputs
        self.loss_function_type = loss_function_type
        self.layers: List[Layer] = []
    
    def add_layer(self, num_neurones, activation_type = 'sigmoid' ):
        num_inputs = self.num_inputs
        if not len(self.layers) == 0:
            num_inputs = self.layers[-1].num_neurones
        layer = Layer(num_neurones, num_inputs, activation_type)
        self.layers.append(layer)
    
    def forward(self, inputs):
        previous_output = inputs
        for layer in self.layers:
            previous_output = layer.forward(previous_output)
        return np.array(previous_output)

    def calculate_loss(self, y_true, y_pred):  #its cost function, not loss function
        if self.loss_function_type == 'binary_corss_entropy':
            return -np.mean((y_true*np.log(y_pred+1e-8))+((1-y_true)*np.log(1-y_pred+1e-8)))
        elif self.loss_function_type == 'mean_squared_error':
            return np.mean((y_true - y_pred)**2)
        else:
            raise NotImplementedError(f"Loss '{self.loss_function_type}' not supported")
        
    def train(self, inputs, y_true, learning_rate):
        y_pred = self.forward(inputs)
        
        output_deltas = y_pred - y_true
        
        deltas = output_deltas
        
        for layer in reversed(self.layers):
            deltas = layer.backward(deltas, learning_rate)
        
        return self.calculate_loss(y_true, y_pred)
    

In [12]:
#testing

my_nn = NeuralNetwork(3)
my_nn.add_layer(4,activation_type='relu')
my_nn.add_layer(2,activation_type='sigmoid')

inputs = [0.9, -0.2, 0.1]
output = my_nn.forward(inputs)

print(output)

[0.53944699 0.50707379]
